> Teorema de Bayes

$P(y \mid x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots, x_n \mid y)}
                                 {P(x_1, \dots, x_n)}$

> Simplificando

$P(y \mid x_1, \dots, x_n) = \frac{P(y) \prod_{i=1}^{n} P(x_i \mid y)}
                                 {P(x_1, \dots, x_n)}$

> Aproximando

$\begin{align}\begin{aligned}P(y \mid x_1, \dots, x_n) \propto P(y) \prod_{i=1}^{n} P(x_i \mid y)\\\Downarrow\\\hat{y} = \arg\max_y P(y) \prod_{i=1}^{n} P(x_i \mid y),\end{aligned}\end{align}$

In [1]:
import pandas as pd

proyectos = pd.read_excel("D:\\data\\ProyectosFinalizados.xlsx", sheet_name="base")

def limpiar_columna(columna):
    # 1. Reemplazar caracteres especiales
    columna = columna.replace(" ", "_") \
        .replace("á", "a") \
        .replace("é", "e") \
        .replace("í", "i") \
        .replace("ó", "o") \
        .replace("ú", "u") \
        .replace("ñ", "n") \
        .replace("+", "_") \
        .replace(".", "_") \
        .replace("(", "_") \
        .replace(")", "")
    import re
    # 2. Poner un guion bajo entre cambio de letras de minúscula a mayúsculas
    columna = re.sub("([a-z])([A-Z])", "\\1_\\2", columna)
    # 3. Poner un guion bajo entre ID y letra
    columna = re.sub("ID([A-Za-z])", "ID_\\1", columna)
    # 4. Poner un guion bajo entre número y letra
    columna = re.sub("([0-9])([A-Za-z])", "\\1_\\2", columna)
    # 4. Poner un guion bajo entre letra y número
    columna = re.sub("([A-Za-z])([0-9])", "\\1_\\2", columna)
    # 5. Quitar un guion bajo entre letra y número
    columna = re.sub("([A-Za-z])_([0-9])", "\\1\\2", columna)
    # 6. Poner un guion bajo entre letra y número final
    columna = re.sub("([A-Za-z])([0-9])$", "\\1_\\2", columna)
    # 7. Convertir a mayúsculas
    columna = columna.upper()
    return columna

proyectos.columns = map(limpiar_columna, proyectos.columns.values)

proyectos.head()

,ID_PROYECTO,EMPRESA,PETICION,PETICION_1,PET_EMPRESA,NOMBRE,CFP,EFICIENCIA,PDR,ESFUERZO_TOTAL_P,...,PDR_S,PDR_M_1,PDR_M_2,PDR_L,LEAD_TIME,F1_GESTIONDELA_DEMANDA,F2_ELICITACION,F3_CONTRUCCION,F4_FINAL,TAMANO
0,1,México,8318.0,8318,8318México,Modificación a la utileria de generación de pa...,12,NaN,33.708333,404.5,...,NaN,NaN,NaN,NaN,914.0,825.0,8.0,42.0,35.0,10-29
1,2,México,9619.0,9619,9619México,Modificacion al programa Vbncarga.exe,2,NaN,147.500000,295.0,...,NaN,NaN,NaN,NaN,609.0,584.0,7.0,14.0,1.0,2-9
2,3,México,10438.0,10438,10438México,Permisos para Cambios de puntos de Sembrado- P...,11,NaN,51.375455,1019.0,...,NaN,NaN,NaN,NaN,519.0,461.0,2.0,37.0,1.0,10-29
3,4,México,10522.0,10522,10522México,Corrección del informe TabuladoCredito,11,NaN,47.727273,525.0,...,NaN,NaN,NaN,NaN,493.0,455.0,9.0,27.0,1.0,10-29
4,5,México,10528.0,10528,10528México,Corrección al Tabulado de Carteras y Hojas men...,4,NaN,26.290000,213.0,...,NaN,NaN,NaN,NaN,557.0,520.0,5.0,30.0,0.0,2-9


In [2]:
# [Apriori >>>] CFP - NUM
# [Apriori] ARQUITECTURA_I - CAT + TEXTO
# [Apriori] LENGUAJE_I - CAT + TEXTO
# [Apriori] BDI - CAT + TEXTO
# [Apriori] FRAMEWORK_I - CAT + TEXTO
# [Priori] METODOLOGIA - CAT + TEXTO
# [Posteriori] EFICIENCIA - DEC

dataset = proyectos[ ["ID_PROYECTO", "CFP", "ARQUITECTURA_I", "LENGUAJE_I", "BDI", "FRAMEWORK_I", "METODOLOGIA", "EFICIENCIA"] ].copy()

import re

def normalizar(cat):
    if str(cat) == "nan":
        return "OTRO"
    return re.sub("\s*", "", str(cat)).upper()

dataset["ARQUITECTURA_I"] = dataset["ARQUITECTURA_I"].map(normalizar)
dataset["LENGUAJE_I"] = dataset["LENGUAJE_I"].map(normalizar)
dataset["BDI"] = dataset["BDI"].map(normalizar).fillna("OTRO")
dataset["FRAMEWORK_I"] = dataset["FRAMEWORK_I"].map(normalizar)
dataset["METODOLOGIA"] = dataset["METODOLOGIA"].map(normalizar)

dataset = dataset.dropna()

dataset.info()

print("ARQUITECTURAS")
print(dataset["ARQUITECTURA_I"].unique())
print("LENGUAJES")
print(dataset["LENGUAJE_I"].unique())
print("BASES DE DATOS")
print(dataset["BDI"].unique())
print("FRAMEWORKS")
print(dataset["FRAMEWORK_I"].unique())
print("METODOLOGÍAS")
print(dataset["METODOLOGIA"].unique())

<class 'pandas.core.frame.DataFrame'>
Index: 1514 entries, 886 to 2954
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID_PROYECTO     1514 non-null   int64  
 1   CFP             1514 non-null   int64  
 2   ARQUITECTURA_I  1514 non-null   object 
 3   LENGUAJE_I      1514 non-null   object 
 4   BDI             1514 non-null   object 
 5   FRAMEWORK_I     1514 non-null   object 
 6   METODOLOGIA     1514 non-null   object 
 7   EFICIENCIA      1514 non-null   float64
dtypes: float64(1), int64(2), object(5)
memory usage: 106.5+ KB
ARQUITECTURAS
['CLIENTE/SERVIDOR' 'CAPAS' 'MVC' 'OTRO']
LENGUAJES
['PHP' 'C++' 'C#' 'COLDFUSION' 'OTRO' 'JAVA' 'VISUALBASIC' 'TYPESCRIPT'
 'OTROS']
BASES DE DATOS
['POSTGRESQL' 'SQLSERVER' 'POSTGRESQL/INFORMIX' 'OTRO' 'INFORMIX' 'VOLTDB'
 'MYSQL' 'DB2']
FRAMEWORKS
['COPPELFRONTWEB' 'OTRO' 'CSHARP' 'RAC(PHP)' 'COPPELFRAMEWORKJAVA(V2.1.2)'
 'CLIENTESWEB(ANGULAR,V1.6.3)' 'SPRINGBOOT(

In [3]:
cfp_mean = dataset["CFP"].mean()

cfp_mean

45.89498018494056

In [4]:
import numpy as np

cfp_25 = np.quantile(dataset["CFP"], 0.25)
cfp_75 = np.quantile(dataset["CFP"], 0.75)

cfp_25, cfp_75

(6.0, 52.0)

In [5]:
eficiencia_mean = dataset["EFICIENCIA"].mean()
eficiencia_25 = np.quantile(dataset["EFICIENCIA"], 0.25)
eficiencia_75 = np.quantile(dataset["EFICIENCIA"], 0.75)

eficiencia_25, eficiencia_mean, eficiencia_75

(0.47862903276785673, 0.8178825563593339, 1.2412652093650638)

In [6]:
premises = [
    [
        ("CFP > mean(CFP)", lambda row: row["CFP"] > cfp_mean),
        ("CFP < mean(CFP)", lambda row: row["CFP"] < cfp_mean),
        ("CFP > q_25(CFP)", lambda row: row["CFP"] > cfp_25),
        ("CFP < q_25(CFP)", lambda row: row["CFP"] < cfp_25),
        ("CFP > q_75(CFP)", lambda row: row["CFP"] > cfp_75),
        ("CFP < q_75(CFP)", lambda row: row["CFP"] < cfp_75),
    ],
    [
        ("ARQUITECTURA=CLIENTE/SERVIDOR", lambda row: row["ARQUITECTURA_I"] == "CLIENTE/SERVIDOR"),
        ("ARQUITECTURA=CAPAS", lambda row: row["ARQUITECTURA_I"] == "CAPAS"),
        ("ARQUITECTURA=MVC", lambda row: row["ARQUITECTURA_I"] == "MVC"),
        ("ARQUITECTURA=OTRO", lambda row: row["ARQUITECTURA_I"] == "OTRO"),
    ],
    [
        ("LENGUAJE=PHP", lambda row: row["LENGUAJE_I"] == "PHP"),
        ("LENGUAJE=C++", lambda row: row["LENGUAJE_I"] == "C++"),
        ("LENGUAJE=C#", lambda row: row["LENGUAJE_I"] == "C#"),
        ("LENGUAJE=COLDFUSION", lambda row: row["LENGUAJE_I"] == "COLDFUSION"),
        ("LENGUAJE=OTRO", lambda row: row["LENGUAJE_I"] == "OTRO"),
        ("LENGUAJE=VISUALBASIC", lambda row: row["LENGUAJE_I"] == "VISUALBASIC"),
        ("LENGUAJE=TYPESCRIPT", lambda row: row["LENGUAJE_I"] == "TYPESCRIPT"),
        ("LENGUAJE=OTROS", lambda row: row["LENGUAJE_I"] == "OTROS"),
    ],
    [
        ("BD=POSTGRESQL", lambda row: row["BDI"] == "POSTGRESQL"),
        ("BD=SQLSERVER", lambda row: row["BDI"] == "SQLSERVER"),
        ("BD=POSTGRESQL/INFORMIX", lambda row: row["BDI"] == "POSTGRESQL/INFORMIX"),
        ("BD=OTRO", lambda row: row["BDI"] == "OTRO"),
        ("BD=INFORMIX", lambda row: row["BDI"] == "INFORMIX"),
        ("BD=VOLTDB", lambda row: row["BDI"] == "VOLTDB"),
        ("BD=MYSQL", lambda row: row["BDI"] == "MYSQL"),
        ("BD=DB2", lambda row: row["BDI"] == "DB2"),
    ],
    [
        ("FRAMEWORK=COPPELFRONTWEB", lambda row: row["FRAMEWORK_I"] == "COPPELFRONTWEB"),
        ("FRAMEWORK=OTRO", lambda row: row["FRAMEWORK_I"] == "OTRO"),
        ("FRAMEWORK=CSHARP", lambda row: row["FRAMEWORK_I"] == "CSHARP"),
        ("FRAMEWORK=RAC(PHP)", lambda row: row["FRAMEWORK_I"] == "RAC(PHP)"),
        ("FRAMEWORK=COPPELFRAMEWORKJAVA(V2.1.2)", lambda row: row["FRAMEWORK_I"] == "COPPELFRAMEWORKJAVA(V2.1.2)"),
        ("FRAMEWORK=CLIENTESWEB(ANGULAR,V1.6.3)", lambda row: row["FRAMEWORK_I"] == "CLIENTESWEB(ANGULAR,V1.6.3)"),
        ("FRAMEWORK=SPRINGBOOT(V2.0)", lambda row: row["FRAMEWORK_I"] == "SPRINGBOOT(V2.0)"),
        ("FRAMEWORK=OTROS", lambda row: row["FRAMEWORK_I"] == "OTROS"),
        ("FRAMEWORK=SPRINGBOOT(V2.0.0)", lambda row: row["FRAMEWORK_I"] == "SPRINGBOOT(V2.0.0)"),
        ("FRAMEWORK=COPPELFRAMEWORKANDROID(JAVAV1.1.0)", lambda row: row["FRAMEWORK_I"] == "COPPELFRAMEWORKANDROID(JAVAV1.1.0)"),
        ("FRAMEWORK=RAC", lambda row: row["FRAMEWORK_I"] == "RAC"),
        ("FRAMEWORK=COPPELFRAMEWORKJAVA", lambda row: row["FRAMEWORK_I"] == "COPPELFRAMEWORKJAVA"),
        ("FRAMEWORK=VUEJS", lambda row: row["FRAMEWORK_I"] == "VUEJS"),
    ],
    [
        ("METODOLOGÍA=CASCADA", lambda row: row["METODOLOGIA"] == "CASCADA"),
        ("METODOLOGÍA=ÁGIL", lambda row: row["METODOLOGIA"] == "ÁGIL"),
    ]
]

conclusions = [
    [
        ("EFICIENCIA > mean(EFICIENCIA)", lambda row: row["EFICIENCIA"] > eficiencia_mean),
        ("EFICIENCIA < mean(EFICIENCIA)", lambda row: row["EFICIENCIA"] < eficiencia_mean),
        ("EFICIENCIA > q_25(EFICIENCIA)", lambda row: row["EFICIENCIA"] > eficiencia_25),
        ("EFICIENCIA < q_25(EFICIENCIA)", lambda row: row["EFICIENCIA"] < eficiencia_25),
        ("EFICIENCIA > q_75(EFICIENCIA)", lambda row: row["EFICIENCIA"] > eficiencia_75),
        ("EFICIENCIA < q_75(EFICIENCIA)", lambda row: row["EFICIENCIA"] < eficiencia_75),
    ]
]

In [10]:
premises = [
    [
        ("CFP > mean(CFP)", lambda row: row["CFP"] > cfp_mean),
        ("CFP < mean(CFP)", lambda row: row["CFP"] < cfp_mean),
    ],
    [
        ("ARQUITECTURA=CLIENTE/SERVIDOR", lambda row: row["ARQUITECTURA_I"] == "CLIENTE/SERVIDOR"),
        ("ARQUITECTURA=CAPAS", lambda row: row["ARQUITECTURA_I"] == "CAPAS"),
        ("ARQUITECTURA=MVC", lambda row: row["ARQUITECTURA_I"] == "MVC"),
        ("ARQUITECTURA=OTRO", lambda row: row["ARQUITECTURA_I"] == "OTRO"),
    ],
    [
        ("LENGUAJE=PHP", lambda row: row["LENGUAJE_I"] == "PHP"),
        ("LENGUAJE=C++", lambda row: row["LENGUAJE_I"] == "C++"),
        ("LENGUAJE=C#", lambda row: row["LENGUAJE_I"] == "C#"),
    ],
    [
        ("BD=POSTGRESQL", lambda row: row["BDI"] == "POSTGRESQL"),
        ("BD=SQLSERVER", lambda row: row["BDI"] == "SQLSERVER"),
        ("BD=POSTGRESQL/INFORMIX", lambda row: row["BDI"] == "POSTGRESQL/INFORMIX"),
        ("BD=MYSQL", lambda row: row["BDI"] == "MYSQL"),
        ("BD=DB2", lambda row: row["BDI"] == "DB2"),
    ],
    [
        ("FRAMEWORK=COPPELFRONTWEB", lambda row: row["FRAMEWORK_I"] == "COPPELFRONTWEB"),
        ("FRAMEWORK=CSHARP", lambda row: row["FRAMEWORK_I"] == "CSHARP"),
        ("FRAMEWORK=RAC(PHP)", lambda row: row["FRAMEWORK_I"] == "RAC(PHP)"),
        ("FRAMEWORK=VUEJS", lambda row: row["FRAMEWORK_I"] == "VUEJS"),
        ("FRAMEWORK=OTRO", lambda row: row["FRAMEWORK_I"] == "OTRO"),
    ],
    [
        ("METODOLOGÍA=CASCADA", lambda row: row["METODOLOGIA"] == "CASCADA"),
        ("METODOLOGÍA=ÁGIL", lambda row: row["METODOLOGIA"] == "ÁGIL"),
    ]
]

conclusions = [
    [
        ("EFICIENCIA > mean(EFICIENCIA)", lambda row: row["EFICIENCIA"] > eficiencia_mean),
        ("EFICIENCIA < mean(EFICIENCIA)", lambda row: row["EFICIENCIA"] < eficiencia_mean),
    ]
]

In [11]:
# Seleccionar 2 premisas y una conclusión

from collections import defaultdict

occurrences = defaultdict(int)
supports = defaultdict(int)
confidences = defaultdict(float)

for premise_i in range(len(premises)):
    for premise_j in range(len(premises)):
        if premise_i == premise_j:
            continue

        sub_premises_i = premises[premise_i]
        sub_premises_j = premises[premise_j]

        for premise_i_label, premise_i_func in sub_premises_i:
            for premise_j_label, premise_j_func in sub_premises_j:
                for sub_conclusions in conclusions:
                    for conclusion_label, conclusion_func in sub_conclusions:
                        need_print = np.random.random() < 0.05

                        if need_print:
                            print("SI {} y {}, ENTONCES, {}".format(premise_i_label, premise_j_label, conclusion_label))

                        for index, row in dataset.iterrows():
                            if premise_i_func(row) and premise_j_func(row):
                                occurrences[(premise_i_label, premise_j_label)] += 1
                                total = occurrences[(premise_i_label, premise_j_label)]
                                if conclusion_func(row):
                                    supports[(premise_i_label, premise_j_label, conclusion_label)] += 1
                                    support = supports[(premise_i_label, premise_j_label, conclusion_label)]

                        support = supports[(premise_i_label, premise_j_label, conclusion_label)]
                        total = occurrences[(premise_i_label, premise_j_label)]

                        if total != 0:
                            confidences[(premise_i_label, premise_j_label, conclusion_label)] = 100 * support / total

                        confidence = confidences[(premise_i_label, premise_j_label, conclusion_label)]

                        if need_print:
                            print("SUPPORT: {} TOTAL: {} CONFIDENCE: {:.2f}".format(support, total, confidence))

SI CFP > mean(CFP) y ARQUITECTURA=MVC, ENTONCES, EFICIENCIA > mean(EFICIENCIA)
SUPPORT: 0 TOTAL: 1 CONFIDENCE: 0.00
SI CFP < mean(CFP) y ARQUITECTURA=CAPAS, ENTONCES, EFICIENCIA > mean(EFICIENCIA)
SUPPORT: 68 TOTAL: 182 CONFIDENCE: 37.36
SI CFP < mean(CFP) y ARQUITECTURA=OTRO, ENTONCES, EFICIENCIA < mean(EFICIENCIA)
SUPPORT: 1 TOTAL: 8 CONFIDENCE: 12.50
SI CFP > mean(CFP) y LENGUAJE=C#, ENTONCES, EFICIENCIA < mean(EFICIENCIA)
SUPPORT: 44 TOTAL: 180 CONFIDENCE: 24.44
SI CFP < mean(CFP) y LENGUAJE=PHP, ENTONCES, EFICIENCIA > mean(EFICIENCIA)
SUPPORT: 135 TOTAL: 321 CONFIDENCE: 42.06
SI CFP < mean(CFP) y FRAMEWORK=COPPELFRONTWEB, ENTONCES, EFICIENCIA > mean(EFICIENCIA)
SUPPORT: 7 TOTAL: 24 CONFIDENCE: 29.17
SI ARQUITECTURA=OTRO y BD=SQLSERVER, ENTONCES, EFICIENCIA < mean(EFICIENCIA)
SUPPORT: 0 TOTAL: 0 CONFIDENCE: 0.00
SI ARQUITECTURA=OTRO y BD=MYSQL, ENTONCES, EFICIENCIA < mean(EFICIENCIA)
SUPPORT: 0 TOTAL: 0 CONFIDENCE: 0.00
SI ARQUITECTURA=CAPAS y FRAMEWORK=RAC(PHP), ENTONCES, EFICIENC

In [14]:
from operator import itemgetter

In [15]:
sorted(occurrences.items(), key=itemgetter(1), reverse=True)

[(('ARQUITECTURA=CLIENTE/SERVIDOR', 'METODOLOGÍA=CASCADA'), 2292),
 (('METODOLOGÍA=CASCADA', 'ARQUITECTURA=CLIENTE/SERVIDOR'), 2292),
 (('FRAMEWORK=OTRO', 'METODOLOGÍA=CASCADA'), 2246),
 (('METODOLOGÍA=CASCADA', 'FRAMEWORK=OTRO'), 2246),
 (('ARQUITECTURA=CLIENTE/SERVIDOR', 'FRAMEWORK=OTRO'), 2062),
 (('FRAMEWORK=OTRO', 'ARQUITECTURA=CLIENTE/SERVIDOR'), 2062),
 (('CFP < mean(CFP)', 'METODOLOGÍA=CASCADA'), 2010),
 (('METODOLOGÍA=CASCADA', 'CFP < mean(CFP)'), 2010),
 (('CFP < mean(CFP)', 'FRAMEWORK=OTRO'), 1842),
 (('FRAMEWORK=OTRO', 'CFP < mean(CFP)'), 1842),
 (('CFP < mean(CFP)', 'ARQUITECTURA=CLIENTE/SERVIDOR'), 1814),
 (('ARQUITECTURA=CLIENTE/SERVIDOR', 'CFP < mean(CFP)'), 1814),
 (('BD=POSTGRESQL', 'METODOLOGÍA=CASCADA'), 1510),
 (('METODOLOGÍA=CASCADA', 'BD=POSTGRESQL'), 1510),
 (('ARQUITECTURA=CLIENTE/SERVIDOR', 'BD=POSTGRESQL'), 1364),
 (('BD=POSTGRESQL', 'ARQUITECTURA=CLIENTE/SERVIDOR'), 1364),
 (('BD=POSTGRESQL', 'FRAMEWORK=OTRO'), 1348),
 (('FRAMEWORK=OTRO', 'BD=POSTGRESQL'), 1

In [16]:
sorted(supports.items(), key=itemgetter(1), reverse=True)

[(('ARQUITECTURA=CLIENTE/SERVIDOR',
   'METODOLOGÍA=CASCADA',
   'EFICIENCIA < mean(EFICIENCIA)'),
  601),
 (('METODOLOGÍA=CASCADA',
   'ARQUITECTURA=CLIENTE/SERVIDOR',
   'EFICIENCIA < mean(EFICIENCIA)'),
  601),
 (('FRAMEWORK=OTRO', 'METODOLOGÍA=CASCADA', 'EFICIENCIA < mean(EFICIENCIA)'),
  590),
 (('METODOLOGÍA=CASCADA', 'FRAMEWORK=OTRO', 'EFICIENCIA < mean(EFICIENCIA)'),
  590),
 (('CFP < mean(CFP)', 'METODOLOGÍA=CASCADA', 'EFICIENCIA < mean(EFICIENCIA)'),
  572),
 (('METODOLOGÍA=CASCADA', 'CFP < mean(CFP)', 'EFICIENCIA < mean(EFICIENCIA)'),
  572),
 (('ARQUITECTURA=CLIENTE/SERVIDOR',
   'METODOLOGÍA=CASCADA',
   'EFICIENCIA > mean(EFICIENCIA)'),
  545),
 (('METODOLOGÍA=CASCADA',
   'ARQUITECTURA=CLIENTE/SERVIDOR',
   'EFICIENCIA > mean(EFICIENCIA)'),
  545),
 (('ARQUITECTURA=CLIENTE/SERVIDOR',
   'FRAMEWORK=OTRO',
   'EFICIENCIA < mean(EFICIENCIA)'),
  537),
 (('FRAMEWORK=OTRO',
   'ARQUITECTURA=CLIENTE/SERVIDOR',
   'EFICIENCIA < mean(EFICIENCIA)'),
  537),
 (('FRAMEWORK=OTRO', '

In [17]:
sorted(confidences.items(), key=itemgetter(1), reverse=True)

[(('CFP > mean(CFP)', 'BD=DB2', 'EFICIENCIA > mean(EFICIENCIA)'), 100.0),
 (('CFP > mean(CFP)', 'FRAMEWORK=VUEJS', 'EFICIENCIA > mean(EFICIENCIA)'),
  100.0),
 (('ARQUITECTURA=CAPAS', 'BD=DB2', 'EFICIENCIA > mean(EFICIENCIA)'), 100.0),
 (('ARQUITECTURA=CAPAS', 'FRAMEWORK=VUEJS', 'EFICIENCIA > mean(EFICIENCIA)'),
  100.0),
 (('LENGUAJE=C++', 'FRAMEWORK=CSHARP', 'EFICIENCIA > mean(EFICIENCIA)'),
  100.0),
 (('BD=DB2', 'CFP > mean(CFP)', 'EFICIENCIA > mean(EFICIENCIA)'), 100.0),
 (('BD=DB2', 'ARQUITECTURA=CAPAS', 'EFICIENCIA > mean(EFICIENCIA)'), 100.0),
 (('BD=POSTGRESQL', 'FRAMEWORK=VUEJS', 'EFICIENCIA > mean(EFICIENCIA)'),
  100.0),
 (('FRAMEWORK=VUEJS', 'CFP > mean(CFP)', 'EFICIENCIA > mean(EFICIENCIA)'),
  100.0),
 (('FRAMEWORK=VUEJS', 'ARQUITECTURA=CAPAS', 'EFICIENCIA > mean(EFICIENCIA)'),
  100.0),
 (('FRAMEWORK=CSHARP', 'LENGUAJE=C++', 'EFICIENCIA > mean(EFICIENCIA)'),
  100.0),
 (('FRAMEWORK=VUEJS', 'BD=POSTGRESQL', 'EFICIENCIA > mean(EFICIENCIA)'),
  100.0),
 (('FRAMEWORK=VUEJS'

In [29]:
for (premise_i_label, premise_j_label, conclusion_label), support in sorted(supports.items(), key=itemgetter(1), reverse=True):

    if premise_i_label > premise_j_label:
        continue

    if support < 10:
        continue

    total = occurrences[(premise_i_label, premise_j_label)]

    if total == 0:
        continue

    confidence = confidences[(premise_i_label, premise_j_label, conclusion_label)]

    if confidence < 60:
        continue

    print("SI {} y {}, ENTONCES, {}".format(premise_i_label, premise_j_label, conclusion_label))

    print("SUPPORT: {} TOTAL: {} CONFIDENCE: {:.2f}".format(support, total, confidence))

SI BD=POSTGRESQL/INFORMIX y CFP > mean(CFP), ENTONCES, EFICIENCIA > mean(EFICIENCIA)
SUPPORT: 20 TOTAL: 64 CONFIDENCE: 62.50
SI CFP > mean(CFP) y FRAMEWORK=CSHARP, ENTONCES, EFICIENCIA > mean(EFICIENCIA)
SUPPORT: 11 TOTAL: 34 CONFIDENCE: 64.71
